# Analýza věku a pohlaví podle pětiletých věkových skupin

### Import knihoven


In [1]:
import pandas as pd
import numpy as np
import pymongo
import utils.data_loader as data_loader
import pprint
import json
from cis import *

### Poznámky

In [2]:
# https://www.czso.cz/csu/rso/uzemne_technicke_jednotky
# https://scitani.gov.cz/vysledky
# https://www.czso.cz/csu/czso/vysledky-scitani-2021-otevrena-data

### Pomocné funkce

In [2]:
# Tisk čísla s oddělovači čárek
def format_with_commas(number):
    return "{:,}".format(number)

# Hezký tisk slovníku pro věk
def print_dictionary_ages(text, dict):
    print(f"{text}")
    for key, value in dict.items():
        print(f"\t=> {key}: {value} ({(value / analysis_result['count'])*100:.2f}%)")

# Hezký tisk slovníku pro věk a pohlaví
def print_dictionary_ages_sex(text, dict):
    print(f"{text}")
    for key, value in dict.items():
        print(f"\t=> {key}:")
        for key, value in value.items():
            print(f"\t\t=> {key}: {value}")

### Uložení CSV dat do MongoDB

In [4]:
data_loader.load_csv_into_db("vek_pohlavi_5", "../data_original/sldb2021_vek5_pohlavi.csv")

### Načtení dat do paměti z MongoDB

In [3]:
data = data_loader.load_from_db("obyvatelstvo_pohlavi_vek_5")

### Analýza

In [4]:
# Pomocné CIS kódy
cis_vek = {
    1100000004: '0 - 4 roky',
    1300050009: '5 - 9 let',
    1300100014: '10 - 14 let',
    1300150019: '15 - 19 let',
    1300200024: '20 - 24 let',
    1300250029: '25 - 29 let',
    1300300034: '30 - 34 let',
    1300350039: '35 - 39 let',
    1300400044: '40 - 44 let',
    1300450049: '45 - 49 let',
    1300500054: '50 - 54 let',
    1300550059: '55 - 59 let',
    1300600064: '60 - 64 let',
    1300650069: '65 - 69 let',
    1300700074: '70 - 74 let',
    1300750079: '75 - 79 let',
    1300800084: '80 - 84 let',
    1300850089: '85 - 89 let',
    1300900094: '90 - 94 let',
    1300950099: '95 - 99 let',
    1201009999: '100 a více let'
}

cis_kraje = {
    3026: 'Středočeský kraj',
    3018: 'Hlavní město Praha',
    3115: 'Jihomoravský kraj',
    3140: 'Moravskoslezský kraj',
    3069: 'Ústecký kraj',
    3034: 'Jihočeský kraj',
    3123: 'Olomoucký kraj',
    3042: 'Plzeňský kraj',
    3131: 'Zlínský kraj',
    3085: 'Královéhradecký kraj',
    3093: 'Pardubický kraj',
    3107: 'Kraj Vysočina',
    3077: 'Liberecký kraj',
    3051: 'Karlovarský kraj'
}

veky_podle_fazi = {
    'deti': ['0 - 4 roky', '5 - 9 let'],
    'dospivani': ['10 - 14 let','15 - 19 let'],
    'dospeli': ['20 - 24 let', '25 - 29 let', '30 - 34 let', '35 - 39 let', '40 - 44 let', '45 - 49 let', '50 - 54 let', '55 - 59 let'],
    'stari': ['65 - 69 let', '70 - 74 let', '75 - 79 let', '80 - 84 let', '85 - 89 let', '95 - 99 let', '100 a více let'],
}

In [5]:
analysis_result = dict()

# Celkový počet obyvatel
analysis_result['count'] = int(data[(data['uzemi_cis'] == 97) & (data['vek_txt'].isnull()) & (data['pohlavi_txt'].isnull())]['hodnota'])

# Celkový počet obyvatel podle pohlaví
analysis_result['count_women'] = int(data[(data['uzemi_cis'] == 97) & (data['vek_txt'].isnull()) & (data['pohlavi_kod'] == 2)]['hodnota'])
analysis_result['count_men'] = int(data[(data['uzemi_cis'] == 97) & (data['vek_txt'].isnull()) & (data['pohlavi_kod'] == 1)]['hodnota'])

# Celkový počet obyvatel podle věku
analysis_result['count_ages'] = {}
for key, value in cis_vek.items():
    analysis_result['count_ages'][value] = int(data[(data['uzemi_cis'] == 97) & (data['vek_kod'] == key) & (data['pohlavi_txt'].isnull())]['hodnota'])

# Celkový počet obyvatel podle věku a pohlaví
analysis_result['count_age_sex'] = {}
for key, value in cis_vek.items():
    analysis_result['count_age_sex'][value] = {}
    analysis_result['count_age_sex'][value]['women'] = int(data[(data['uzemi_cis'] == 97) & (data['vek_kod'] == key) & (data['pohlavi_kod'] == 2)]['hodnota'])
    analysis_result['count_age_sex'][value]['men'] = int(data[(data['uzemi_cis'] == 97) & (data['vek_kod'] == key) & (data['pohlavi_kod'] == 1)]['hodnota'])

# Celkový počet obyvatel podle věkových fází (období)
analysis_result['count_age_phase'] = {}
for key, value in veky_podle_fazi.items():
    analysis_result['count_age_phase'][key] = 0
    for phase in value:
        analysis_result['count_age_phase'][key] += int(data[(data['uzemi_cis'] == 97) & (data['vek_txt'] == phase) & (data['pohlavi_txt'].isnull())]['hodnota'])

# Celkový počet obyvatel podle věkových fází (období) a pohlaví
analysis_result['count_age_phase_sex'] = {}
for key, value in veky_podle_fazi.items():
    analysis_result['count_age_phase_sex'][key] = {}
    analysis_result['count_age_phase_sex'][key]['women'] = 0
    analysis_result['count_age_phase_sex'][key]['men'] = 0
    for phase in value:
        analysis_result['count_age_phase_sex'][key]['women'] += int(data[(data['uzemi_cis'] == 97) & (data['vek_txt'] == phase) & (data['pohlavi_kod'] == 2)]['hodnota'])
        analysis_result['count_age_phase_sex'][key]['men'] += int(data[(data['uzemi_cis'] == 97) & (data['vek_txt'] == phase) & (data['pohlavi_kod'] == 1)]['hodnota'])

# Celkový počet obyvatel podle okresu, věku a pohlaví
analysis_result['count_sex_district'] = {}
for key, value in cis_okresy.items():
    key_int = int(key)
    analysis_result['count_sex_district'][key_int] = {}
    analysis_result['count_sex_district'][key_int]['name'] = value
    analysis_result['count_sex_district'][key_int]['data'] = {}
    analysis_result['count_sex_district'][key_int]['data']['women'] = []
    analysis_result['count_sex_district'][key_int]['data']['men'] = []
    for key2, value2 in cis_vek.items(): 
        analysis_result['count_sex_district'][key_int]['data']['women'].append(int(data[(data['uzemi_kod'] == key_int) & (data['uzemi_cis'] == 101) & (data['pohlavi_kod'] == 2) & (data['vek_txt'] == value2)]['hodnota']))
        analysis_result['count_sex_district'][key_int]['data']['men'].append(int(data[(data['uzemi_kod'] == key_int) & (data['uzemi_cis'] == 101) & (data['pohlavi_kod'] == 1) & (data['vek_txt'] == value2)]['hodnota']))

# Výsledky analýzy

In [8]:
# Celkový počet obyvatel
print(f"Celkový počet obyvatel: {format_with_commas(analysis_result['count'])}")
print(f"\nCelkový počet žen: {format_with_commas(analysis_result['count_women'])} ({(analysis_result['count_women'] / analysis_result['count'])*100:.2f}%)")
print(f"\nCelkový počet muž: {format_with_commas(analysis_result['count_men'])} ({(analysis_result['count_men'] / analysis_result['count'])*100:.2f}%)")
print_dictionary_ages("\nCelkový počet obyvatel podle věku:", analysis_result['count_ages'])
print_dictionary_ages_sex("\nCelkový počet obyvatel podle věku a pohlaví:", analysis_result['count_age_sex'])
print_dictionary_ages("\nCelkový počet obyvatel podle věkoých fází:", analysis_result['count_age_phase'])
print_dictionary_ages_sex("\nCelkový počet obyvatel podle věkových fází a pohlaví:", analysis_result['count_age_phase_sex'])
# print_dictionary_ages_sex("\nCelkový počet obyvatel podle pohlaví a kraje:", analysis_result['count_sex_district'])
# pprint.pprint(analysis_result['count_sex_district'])
# output = json.dumps(analysis_result['count_sex_district'], ensure_ascii=False).encode('utf8')
# with open('output.json', 'w') as f:
#     f.write(output.decode())

Celkový počet obyvatel: 10,524,167

Celkový počet žen: 5,337,619 (50.72%)

Celkový počet muž: 5,186,548 (49.28%)

Celkový počet obyvatel podle věku:
	=> 0 - 4 roky: 560632 (5.33%)
	=> 5 - 9 let: 550024 (5.23%)
	=> 10 - 14 let: 581104 (5.52%)
	=> 15 - 19 let: 489836 (4.65%)
	=> 20 - 24 let: 477063 (4.53%)
	=> 25 - 29 let: 601168 (5.71%)
	=> 30 - 34 let: 690561 (6.56%)
	=> 35 - 39 let: 719089 (6.83%)
	=> 40 - 44 let: 857917 (8.15%)
	=> 45 - 49 let: 877616 (8.34%)
	=> 50 - 54 let: 688110 (6.54%)
	=> 55 - 59 let: 666391 (6.33%)
	=> 60 - 64 let: 616608 (5.86%)
	=> 65 - 69 let: 668081 (6.35%)
	=> 70 - 74 let: 618629 (5.88%)
	=> 75 - 79 let: 419368 (3.98%)
	=> 80 - 84 let: 243373 (2.31%)
	=> 85 - 89 let: 135311 (1.29%)
	=> 90 - 94 let: 52786 (0.50%)
	=> 95 - 99 let: 9853 (0.09%)
	=> 100 a více let: 647 (0.01%)

Celkový počet obyvatel podle věku a pohlaví:
	=> 0 - 4 roky:
		=> women: 273743
		=> men: 286889
	=> 5 - 9 let:
		=> women: 267952
		=> men: 282072
	=> 10 - 14 let:
		=> women: 283743
